In [137]:
import numpy as np
import matplotlib.pyplot as plt

In [138]:
from loaders import clean_text_for_pattern_letter
from temp_text_puh import template_str
# input_str = clean_text_for_pattern_letter(template_str, out_len=len(template_str))
input_str = clean_text_for_pattern_letter(template_str, out_len=3000)
print(len(input_str))
letters = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u',
               'v', 'w', 'x', 'y', 'z', ' ', '.', ',', '"', '\'', '1']
print(f'{letters=}')

len(letters)=32
3000
letters=['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', ' ', '.', ',', '"', "'", '1']


In [139]:
def predict_train(input_str,
                  letters,
                  max_history=4,
                  max_index: int = 4,):
    WIN: dict[str, dict[str, int]] = {}

    HistoryHW: list[dict[str, int]] = []
    HistoryW: list[dict[str, dict[str, dict[int, int]]]] = []

    for h_idx in range(max_history):
        HistoryHW.append({})
        HistoryW.append({})

    IDX: list = []

    for letter in letters:
        WIN.setdefault(letter, {})

        for idx in range(max_index):
            WIN[letter].setdefault(idx, 0)

    prev_indices = [0]

    for inp_idx, inp in enumerate(input_str):
        if inp_idx <= 0:
            WIN[inp][prev_indices[0]] += 1
            continue

        IDX.append(prev_indices[0])

        IdxO = {}

        for h_idx in range(max_history):
            prev_idx = inp_idx - 1 - h_idx
            if prev_idx < 0:
                break

            prev_inp = input_str[prev_idx]

            HW = HistoryHW[h_idx]
            W = HistoryW[h_idx]

            key_HW = prev_inp + str(prev_indices[h_idx]) + ':' + inp

            if HW.get(key_HW, -1) != -1:
                inp_has_index = HW[key_HW]

                IdxO.setdefault(inp_has_index, 0)
                IdxO[inp_has_index] += 1

        new_index = 0
        if len(IdxO) == 0:
            new_index = min(WIN[inp], key=WIN[inp].get)
            WIN[inp][new_index] += 1
        else:
            new_index = max(IdxO, key=IdxO.get)

        for h_idx in range(max_history):
            prev_idx = inp_idx - 1 - h_idx
            if prev_idx < 0:
                break

            prev_inp = input_str[prev_idx]

            key_I = prev_inp + str(prev_indices[h_idx])
            HistoryW[h_idx].setdefault(key_I, {})
            HistoryW[h_idx][key_I].setdefault(inp, {})
            HistoryW[h_idx][key_I][inp].setdefault(new_index, 0)
            HistoryW[h_idx][key_I][inp][new_index] += 1

            key_HW = prev_inp + str(prev_indices[h_idx]) + ':' + inp

            HistoryHW[h_idx].setdefault(key_HW, new_index)

        # recurtion
        prev_indices.insert(0, new_index)
        if len(prev_indices) > max_history:
            prev_indices = prev_indices[:-1]

    return HistoryW, HistoryHW, IDX

In [140]:
def predict(input_str: str, 
            HistoryW: list[dict[str, dict[str, dict[int, int]]]], 
            max_history = 4,
            max_index = 4,):
    err = 0
    TF = []
    output = [input_str[0]]

    history_indices = []
    history_indices.append([x for x in range(max_index)])

    for input_idx, input in enumerate(input_str[:-1]):
        HO: dict[str, dict[int, int]] = {}

        for h_idx in range(max_history):
            inp_idx = input_idx - h_idx
            if inp_idx < 0:
                break
            # print(f'{h_idx=} {inp_idx=}')
            W = HistoryW[h_idx]
            inp = input_str[inp_idx]
            indices = history_indices[h_idx]

            key_I = inp + str(indices[0])
            O = W.get(key_I, {})
            # print(f'{O=}')
            # print()

            for index in indices[1:]:
                key_I = inp + str(index)

                index_O = W.get(key_I, {})
                # print(f'{index_O=}')
                # print()

                for key, val in index_O.items():
                    O.setdefault(key, {})
                    O[key].update(val)
            
            # print(f'{O=}')
            # print()
            
            for key_O, val_O in O.items():
                HO.setdefault(key_O, {})
                
                for key_index, val_index in val_O.items():
                    HO[key_O].setdefault(key_index, 0)

                    HO[key_O][key_index] += val_index

        # print(f'{HO=}')
        # print()

        length = 0
        for v in HO.values():
            length += len(v)

        # tf = length
        tf = len(HO)

        TF.append(tf)

        max_symbol = 'a'
        max_symbol_index = 0
        max_val = 0
        for key, val in HO.items():
            max_idx = max(val, key=val.get)
            HO[key] = {max_idx: val[max_idx]}

            if val[max_idx] > max_val:
                max_val = val[max_idx]
                max_symbol_index = max_idx
                max_symbol = key

        # print(f'{HO=}')
        next_inp = input_str[input_idx + 1]
        print(f'{max_symbol=}', f'{next_inp=}', max_symbol_index, max_val)

        new_indices = [max_symbol_index]

        output.append(max_symbol)

        # err
        if max_symbol != next_inp:
            err += 1

            index_stats = HO.get(next_inp, 0)
            
            if index_stats == 0:
                new_indices = [x for x in range(max_index)]
                print(f'{new_indices=}')
            else:
                max_symbol_index = list(index_stats.keys())[0]

                new_indices = [max_symbol_index]
        
        # recurtion
        history_indices.insert(0, new_indices)
        if len(history_indices) > max_history:
            history_indices = history_indices[:-1]

    return err, TF, output

In [141]:
def get_word_stats(input_str: str | list, TF: list, ):
    buffer: list = [input_str[0]]
    WordStats: dict[str, int] = {}

    prev_tf = TF[0]

    for inp_idx, inp in enumerate(input_str[:-1]):
        if inp_idx <= 0:
            continue

        if TF[inp_idx] - prev_tf > 0.0: # and abs(TF[inp_idx] - prev_tf) > 0.2:
            word = ''.join(buffer)

            WordStats.setdefault(word, 0)
            WordStats[word] += 1

            buffer = [inp]
            prev_tf = TF[inp_idx]
            
            continue

        buffer.append(inp)
        prev_tf = TF[inp_idx]
    
    # sort
    WordStats = dict(sorted(WordStats.items(), key=lambda item: item[1], reverse=True))

    return WordStats

In [142]:
def show_TF(TF:list, begin = 100, length = 400):
    arr_TF = np.array(TF, dtype=float)

    arr_TF = np.pad(arr_TF, (1, 0))[:-1] - arr_TF
    arr_TF[arr_TF > 0] = 0
    arr_TF = np.abs(arr_TF)

    arr_TF = arr_TF[begin:length]

    fig, axs = plt.subplots(1, 1, figsize=(50, 5), tight_layout=True)
    axs.plot(np.arange(arr_TF.size), arr_TF)
    axs.xaxis.set_ticks(np.arange(len(input_str[begin:length])),input_str[begin:length])
    axs.xaxis.grid(True)

    plt.show()

In [143]:
def get_upstream_input(input_str: str, TF: list):
    upstream_inp: dict[int, list] = {}
    
    temp_seq = []

    prev_tf = TF[0]

    for inp_idx, inp in enumerate(input_str[1:-1]):
        tf = TF[inp_idx]
        
        if tf > prev_tf:
            upstream_inp.setdefault(inp_idx, temp_seq.copy())
            temp_seq.clear()

        temp_seq.append(inp)

        prev_tf = tf

    return upstream_inp

In [144]:
def get_upstream_patterns(upstream_input: list, letters: list):
    letters_dict: dict[str, int] = {}
    for let_idx, letter in enumerate(letters):
        letters_dict.setdefault(letter, let_idx)

    upstream_patterns = []

    for upstream_letters in upstream_input.values():
        upstream_pattern = np.zeros(len(letters))

        for letter in upstream_letters:
            pattern_idx = letters_dict.get(letter, 0)

            upstream_pattern[pattern_idx] = 1
        
        upstream_patterns.append(upstream_pattern)

    upstream_patterns = np.array(upstream_patterns)

    return upstream_patterns

In [145]:
def train(upstream_patterns, layer_size = 32):
    I = np.zeros(upstream_patterns.shape[1])
    O = np.zeros(layer_size)
    W = np.ones((O.size, I.size))
    Trace = np.zeros((O.size, I.size))
    print('W', W.shape, W.max(), W.min(), np.median(W))

    k = 1
    push_delta = 0.4

    prec = 1e-30

    epoch_cnt = 100
    eps0 = 0.02

    eps = 0.05
    rate_drop = 0.0

    AVG_O = np.zeros(O.shape)
    AVG_O_CNT = np.zeros(O.shape)

    samples_idx = np.arange(upstream_patterns.shape[0])

    for epoch_idx in range(epoch_cnt):
        np.random.shuffle(samples_idx)

        for i_idx in samples_idx:
            I = upstream_patterns[i_idx]

            # infer
            O = np.dot(W, I)

            # train
            winner_ids = np.argsort(O)[::-1][:1+k]
            winner = winner_ids[0]
            # loosers = winner_ids[1:1+k]

            # update
            W[winner] = (1 - eps) * W[winner] + eps * (I - 0.5) + Trace[winner]*eps
            W[winner] = np.clip(W[winner], -1, 1)

            # W[loosers] = (1 - eps) * W[loosers] + (eps * (I - 0.5) * -push_delta) + Trace[loosers]*eps
            # W[loosers] = np.clip(W[loosers], -1, 1)

            # update Trace
            rate_Trace = np.zeros(I.shape)
            rate_Trace[(I > 0.5)] = 0.3
            rate_Trace[(I < 0.5)] = -0.001

            dTrace = np.zeros(W.shape)
            dTrace[winner] = rate_Trace
            Trace += dTrace
            Trace = np.clip(Trace, 0, 1)

            eps -= eps * rate_drop
    
    return W

In [146]:
def get_next_layer_sequence(upstream_patterns, W):
    sequence = []

    for pattern in upstream_patterns:
        O = np.dot(W, pattern)

        winner = np.argmax(O)

        sequence.append(str(winner))

    return sequence

In [147]:
def downstream_train(input: list, next_input: list, upstream_inp: dict[int, list]):
    DSStats = {}

    ds_idx = 0
    ds_inp = '0'

    for inp_idx, inp in enumerate(input[1:]):
        has_ds_input = upstream_inp.get(inp_idx)
        if has_ds_input:
            ds_inp = next_input[ds_idx]
            ds_idx += 1

        ds_key = ds_inp + ':' + inp
        DSStats.setdefault(ds_key, 0)
        DSStats[ds_key] += 1
    
    return DSStats

In [148]:
def get_letters(W):
    letters = []

    for let_idx in range(W.shape[0]):
        letters.append(str(let_idx))

    return letters

In [149]:
def downstream_predict(input_str: str, 
            HistoryW: list[dict[str, dict[str, dict[int, int]]]], 
            upstream_inp: dict[int, list],
            DSStats: dict[str, int],
            ds_input: list,
            max_history = 4,
            max_index = 4,):
    err = 0
    TF = []
    out = [input_str[0]]

    history_indices = []
    history_indices.append([x for x in range(max_index)])

    ds_idx = 0
    ds_inp = '0'

    for input_idx, input in enumerate(input_str[:-1]):
        HO: dict[str, dict[int, int]] = {}

        for h_idx in range(max_history):
            inp_idx = input_idx - h_idx
            if inp_idx < 0:
                break
            # print(f'{h_idx=} {inp_idx=}')
            W = HistoryW[h_idx]
            inp = input_str[inp_idx]
            indices = history_indices[h_idx]

            key_I = inp + str(indices[0])
            O = W.get(key_I, {})
            # print(f'{O=}')
            # print()

            for index in indices[1:]:
                key_I = inp + str(index)

                index_O = W.get(key_I, {})
                # print(f'{index_O=}')
                # print()

                for key, val in index_O.items():
                    O.setdefault(key, {})
                    O[key].update(val)
            
            # print(f'{O=}')
            # print()
            
            for key_O, val_O in O.items():
                HO.setdefault(key_O, {})
                
                for key_index, val_index in val_O.items():
                    HO[key_O].setdefault(key_index, 0)

                    HO[key_O][key_index] += val_index

        # print(f'{HO=}')
        # print()

        length = 0
        for v in HO.values():
            length += len(v)

        # tf = length
        tf = len(HO)

        TF.append(tf)

        # ds input
        has_ds_input = upstream_inp.get(inp_idx)
        if has_ds_input:
            ds_inp = ds_input[ds_idx]
            ds_idx += 1

        # max symbol
        max_symbol = 'a'
        max_symbol_index = 0
        max_val = 0
        for key, val in HO.items():
            max_idx = max(val, key=val.get)
            HO[key] = {max_idx: val[max_idx]}
            max_idx_val = val[max_idx]

            # ds predict
            ds_key = ds_inp + ':' + key
            ds_stats = DSStats.get(ds_key)
            # if ds_stats:
            #     max_idx_val += ds_stats

            if max_idx_val > max_val and ds_stats:
                max_val = max_idx_val
                max_symbol_index = max_idx
                max_symbol = key

        out.append(max_symbol)

        # print(f'{HO=}')
        next_inp = input_str[input_idx + 1]
        print(f'{max_symbol=}', f'{next_inp=}', max_symbol_index, max_val)

        new_indices = [max_symbol_index]

        # err
        if max_symbol != next_inp:
            err += 1

            index_stats = HO.get(next_inp, 0)
            
            if index_stats == 0:
                new_indices = [x for x in range(max_index)]
                print(f'{new_indices=}')
            else:
                max_symbol_index = list(index_stats.keys())[0]

                new_indices = [max_symbol_index]
        
        # recurtion
        history_indices.insert(0, new_indices)
        if len(history_indices) > max_history:
            history_indices = history_indices[:-1]

    return err, TF, out

# print(f'{err_L1=}', err_L1 / (len(input_str) / 100))
# print(f'{err_L1_ds=}', err_L1_ds / (len(input_str) / 100))

In [150]:
def upstream_multylayer_train(input_str: str | list,
                              letters: list,
                              layers_cnt=3,
                              max_index=32,
                              max_history=1,
                              layer_size=128,):
    input_symbols = input_str
    lttrs = letters

    layers: list[dict] = []
    errors = []

    for layer_idx in range(layers_cnt):
        # print(f'{lttrs=}')
        # print(f'{input_symbols=}')
        HistoryW, HistoryHW, IDX = predict_train(input_str=input_symbols,
                                                 letters=lttrs,
                                                 max_history=max_history,
                                                 max_index=max_index)
        err, TF, out = predict(input_symbols, HistoryW, max_index=max_index, max_history=max_history)
        print(f'{layer_idx=} {err=}')
        errors.append(err)
        # print(f'{TF=}')
        # print(f'{out=}')

        upstream_input = get_upstream_input(input_str=input_symbols, TF=TF)
        upstream_patterns = get_upstream_patterns(upstream_input=upstream_input, letters=lttrs)

        # print(f'{upstream_patterns=}')
        if upstream_patterns.size > 0:
            W = train(upstream_patterns, layer_size=layer_size)
        else:
            W = np.ones((layer_size, 1))

        # out
        layer = {}
        layer.setdefault('upstream_input', upstream_input)
        layer.setdefault('out', out)
        layer.setdefault('HistoryW', HistoryW)
        layer.setdefault('W', W)
        layer.setdefault('letters', lttrs)
        layer.setdefault('input_symbols', input_symbols)
        layers.append(layer)

        # recur
        input_symbols = get_next_layer_sequence(upstream_patterns, W)
        lttrs = get_letters(W)

    return layers, errors

layers_cnt = 3
layer_size=128*4

layers, errors = upstream_multylayer_train(input_str,
                                   letters,
                                   layers_cnt=layers_cnt,
                                   max_index=32,
                                   max_history=1,
                                   layer_size=layer_size,)

# print(f'{len(layers)=}')
# for layer in layers:
#     for k, v in layer.items():
#         print(k, v)

print(f'{errors[::-1]=}')


max_symbol='h' next_inp='h' 2 2
max_symbol='r' next_inp='a' 25 2
max_symbol='s' next_inp='p' 10 3
new_indices=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]
max_symbol='o' next_inp='t' 2 3
max_symbol='h' next_inp='e' 28 5
max_symbol=' ' next_inp='r' 17 3
max_symbol=' ' next_inp=' ' 0 2
max_symbol='h' next_inp='i' 31 5
max_symbol='n' next_inp='n' 0 4
max_symbol=' ' next_inp=' ' 1 2
max_symbol='i' next_inp='w' 10 3
max_symbol='h' next_inp='h' 1 1
max_symbol='i' next_inp='i' 1 3
max_symbol=' ' next_inp='c' 12 2
max_symbol='h' next_inp='h' 2 1
max_symbol='r' next_inp=' ' 25 2
max_symbol='w' next_inp='w' 1 5
max_symbol='e' next_inp='e' 1 3
max_symbol=' ' next_inp=' ' 3 8
max_symbol='a' next_inp='a' 1 5
max_symbol='n' next_inp='r' 21 3
max_symbol='e' next_inp='e' 2 2
max_symbol=' ' next_inp=' ' 4 9
max_symbol='a' next_inp='i' 26 4
max_symbol='s' next_inp='n' 9 3
max_symbol=' ' next_inp='t' 23 3
max_symbol=' ' next_inp='r

In [151]:
def downstream_multylayer_train(layers: list[dict]):
    layers_idx = [x for x in range(len(layers) - 1)]
    layers_idx = layers_idx[::-1]

    for layer_idx in layers_idx:
        layer = layers[layer_idx]
        top_layer = layers[layer_idx + 1]

        ds_stats = downstream_train(input=layer['input_symbols'], 
                                    next_input=top_layer['input_symbols'], 
                                    upstream_inp=layer['upstream_input'],)
        
        layers[layer_idx].setdefault('ds_stats', ds_stats)
        
    return layers

layers = downstream_multylayer_train(layers)

for layer in layers:
    print(f'{layer.get("ds_stats")=}')

layer.get("ds_stats")={'0:h': 1, '0:a': 1, '0:p': 1, '162:t': 1, '162:e': 1, '162:r': 1, '162: ': 1, '359:i': 1, '359:n': 1, '318: ': 13, '3:w': 14, '3:h': 44, '3:i': 38, '498:c': 1, '498:h': 1, '36: ': 1, '3:e': 45, '3: ': 71, '134:a': 2, '134:r': 1, '134:e': 2, '254: ': 4, '3:n': 25, '3:t': 49, '3:r': 16, '3:o': 59, '175:d': 2, '175:u': 2, '175:c': 2, '175:e': 3, '179:d': 2, '179: ': 6, '8:t': 6, '8:o': 7, '28: ': 7, '336:n': 13, '336:i': 5, '336:e': 4, '336: ': 2, '318:t': 1, '318:h': 1, '318:e': 3, '95: ': 43, '3:p': 14, '89:o': 8, '89:h': 6, '89: ': 6, '17:a': 2, '17:n': 2, '71:d': 4, '71: ': 14, '8:s': 1, '97:m': 7, '97:e': 7, '77: ': 13, '3:b': 15, '74:e': 7, '74:s': 4, '74: ': 10, '275:a': 4, '275:n': 2, '8:h': 12, '95:e': 8, '0: ': 18, '3:s': 24, '64:r': 2, '64:i': 1, '2:e': 11, '2:s': 10, '2: ': 13, '275:b': 1, '275:e': 4, '74:g': 4, '74:i': 4, '222:n': 8, '5: ': 20, '72:r': 6, '72:e': 2, '231: ': 13, '228:e': 6, '228:d': 3, '228:w': 3, '228:a': 16, '146:r': 1, '146:d': 1, '1

In [152]:
def downstream_multylayer_predict(layers: list[dict], max_index=32, max_history=1, ):
    layers_idx = [x for x in range(len(layers) - 1)]
    layers_idx = layers_idx[::-1]

    errors = []

    for layer_idx in layers_idx:
        layer = layers[layer_idx]
        top_layer = layers[layer_idx + 1]

        err_ds, TF_ds, out_ds = downstream_predict(input_str=layer['input_symbols'],
                                                   HistoryW=layer['HistoryW'],
                                                   upstream_inp=layer['upstream_input'],
                                                   DSStats=layer['ds_stats'],
                                                   ds_input=top_layer['out'],
                                                   max_history=max_history,
                                                   max_index=max_index, )

        print(f'{layer_idx=} {err_ds=}')

        layers[layer_idx]['out'] = out_ds

        errors.append(err_ds)

    return errors

errors = downstream_multylayer_predict(layers)
print(f'{errors=}')

max_symbol='359' next_inp='359' 0 2
max_symbol='318' next_inp='318' 0 2
max_symbol='3' next_inp='3' 0 7
max_symbol='3' next_inp='498' 1 3
max_symbol='36' next_inp='36' 0 2
max_symbol='3' next_inp='3' 1 3
max_symbol='134' next_inp='134' 0 2
max_symbol='254' next_inp='254' 0 2
max_symbol='3' next_inp='3' 2 2
max_symbol='175' next_inp='175' 0 2
max_symbol='179' next_inp='179' 0 2
max_symbol='8' next_inp='8' 0 2
max_symbol='28' next_inp='28' 0 2
max_symbol='3' next_inp='3' 3 2
max_symbol='336' next_inp='336' 0 2
max_symbol='318' next_inp='318' 1 2
max_symbol='95' next_inp='95' 0 2
max_symbol='3' next_inp='3' 4 3
max_symbol='14' next_inp='89' 17 5
max_symbol='17' next_inp='17' 0 3
max_symbol='71' next_inp='71' 0 4
max_symbol='8' next_inp='8' 1 2
max_symbol='97' next_inp='97' 0 2
max_symbol='77' next_inp='77' 0 3
max_symbol='3' next_inp='3' 5 3
max_symbol='71' next_inp='74' 14 3
max_symbol='275' next_inp='275' 0 2
max_symbol='71' next_inp='71' 1 2
max_symbol='8' next_inp='8' 2 2
max_symbol='

In [153]:
layer_cnt = 3

max_index_L1 = 32
max_history_L1 = 1

layer_size = 32*4

for layer_idx in range(layer_cnt):
    HistoryW, HistoryHW, IDX = predict_train(input_str, letters, max_history=max_history_L1, max_index=max_index_L1)

    err_L1, TF_L1, out_L1 = predict(input_str, HistoryW, max_index=max_index_L1, max_history=max_history_L1)

    upstream_input = get_upstream_input(input_str=input_str, TF=TF_L1)
    upstream_patterns = get_upstream_patterns(upstream_input=upstream_input, letters=letters)

    W = train(upstream_patterns, layer_size=layer_size)

    input_str_L2 = get_next_layer_sequence(upstream_patterns, W)

    ds_stats = downstream_train(input=input_str, next_input=input_str_L2, upstream_inp=upstream_input)

    max_index_L2 = 32
    max_history_L2 = 1
    letters_L2 = get_letters(W)

    HistoryW_L2, HistoryHW_L2, IDX_L2 = predict_train(input_str_L2, 
                                                    letters_L2, 
                                                    max_history=max_history_L2, 
                                                    max_index=max_index_L2,)

    err_L2, TF_L2, out_L2 = predict(input_str_L2, HistoryW_L2, max_index=max_index_L2, max_history=max_history_L2)

    err_L1_ds, TF_L1_ds, out_L1_ds = downstream_predict(input_str=input_str, 
                                                        HistoryW=HistoryW, 
                                                        upstream_inp=upstream_input,
                                                        DSStats=ds_stats,
                                                        ds_input=out_L2,
                                                        max_history=max_history_L1,
                                                        max_index=max_index_L1, )

max_symbol='h' next_inp='h' 2 2
max_symbol='r' next_inp='a' 25 2
max_symbol='s' next_inp='p' 10 3
new_indices=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]
max_symbol='o' next_inp='t' 2 3
max_symbol='h' next_inp='e' 28 5
max_symbol=' ' next_inp='r' 17 3
max_symbol=' ' next_inp=' ' 0 2
max_symbol='h' next_inp='i' 31 5
max_symbol='n' next_inp='n' 0 4
max_symbol=' ' next_inp=' ' 1 2
max_symbol='i' next_inp='w' 10 3
max_symbol='h' next_inp='h' 1 1
max_symbol='i' next_inp='i' 1 3
max_symbol=' ' next_inp='c' 12 2
max_symbol='h' next_inp='h' 2 1
max_symbol='r' next_inp=' ' 25 2
max_symbol='w' next_inp='w' 1 5
max_symbol='e' next_inp='e' 1 3
max_symbol=' ' next_inp=' ' 3 8
max_symbol='a' next_inp='a' 1 5
max_symbol='n' next_inp='r' 21 3
max_symbol='e' next_inp='e' 2 2
max_symbol=' ' next_inp=' ' 4 9
max_symbol='a' next_inp='i' 26 4
max_symbol='s' next_inp='n' 9 3
max_symbol=' ' next_inp='t' 23 3
max_symbol=' ' next_inp='r